# A2A Orchestrator Building Blocks

This notebook contains all the building blocks to create an A2A-compliant orchestrator agent.

## What is an Orchestrator?

An orchestrator is a special type of agent that:
- **Understands** natural language requests
- **Plans** execution strategies
- **Coordinates** multiple specialized agents
- **Synthesizes** results from multiple sources
- **Retransmits** status updates to users

## Key Differences from Regular Agents

1. **Uses tools** to plan and coordinate (not just process)
2. **Calls other agents** via `call_agent()` method
3. **Manages complex workflows** across multiple agents
4. **Synthesizes multiple Artifacts** into comprehensive responses
5. **Handles partial failures** gracefully

## How to Use This Notebook

1. **Copy the code cells** for your orchestrator
2. **Modify** the agent list and coordination logic
3. **Duplicate tool cells** for additional orchestration tools
4. **Customize** the system instruction for your domain

## 1. Setup and Imports

Orchestrator-specific imports including tools and task management.

In [ ]:
#!/usr/bin/env python3
"""
Orchestrator Agent - Coordinates multiple agents to complete complex tasks.
Fully A2A-compliant orchestrator following specification v0.3.0.
"""

import os
import sys
import json
import asyncio
from pathlib import Path
from typing import List, Dict, Any, Optional, Union
from datetime import datetime

# Add parent directories to path
sys.path.insert(0, str(Path(__file__).parent.parent))

# Core A2A imports
from base import A2AAgent
from a2a.types import AgentSkill
from utils.logging import get_logger
from utils.a2a_client import A2AClient

# For tools
from google.adk.tools import FunctionTool

logger = get_logger(__name__)

## 2. Base Orchestrator Class

The orchestrator extends A2AAgent but focuses on coordination rather than direct processing.

In [ ]:
class OrchestratorAgent(A2AAgent):
    """
    LLM-powered orchestrator that coordinates multiple agents.
    
    The orchestrator:
    - Uses tools to understand, plan, and synthesize
    - Calls other agents via call_agent()
    - Manages complex multi-agent workflows
    - Returns synthesized Artifacts from all agents
    """
    
    def __init__(self):
        """Initialize the orchestrator."""
        super().__init__()
        
        # Track execution state
        self.execution_history = []
        self.agent_registry = self._load_agent_registry()
    
    def _load_agent_registry(self) -> Dict[str, str]:
        """Load available agents from config."""
        try:
            with open('config/agents.json', 'r') as f:
                config = json.load(f)
                return config.get('agents', {})
        except Exception as e:
            logger.warning(f"Could not load agent registry: {e}")
            return {
                # Default agents
                "analyzer": "http://analyzer:8000",
                "processor": "http://processor:8000",
                "validator": "http://validator:8000"
            }

## 3. Orchestrator Metadata

Define the orchestrator's identity and capabilities.

In [ ]:
    def get_agent_name(self) -> str:
        """Return orchestrator name."""
        return "Multi-Agent Orchestrator"
    
    def get_agent_description(self) -> str:
        """Return detailed orchestrator description."""
        return (
            "LLM-powered orchestrator that coordinates multiple specialized agents "
            "to complete complex tasks. Understands natural language requests, "
            "plans execution strategies, manages agent coordination, handles errors, "
            "and synthesizes results into comprehensive responses."
        )
    
    def get_agent_version(self) -> str:
        return "2.0.0"
    
    def get_agent_skills(self) -> List[AgentSkill]:
        """Declare orchestration capabilities."""
        return [
            AgentSkill(
                id="orchestration",
                name="Multi-Agent Orchestration",
                description="Coordinate multiple agents to complete complex tasks",
                tags=["orchestration", "coordination", "workflow", "multi-agent"],
                examples=[
                    "Analyze this document using all available agents",
                    "Process this data through the validation pipeline",
                    "Coordinate analysis and generate a report"
                ],
                inputModes=["text/plain", "application/json"],
                outputModes=["application/json"]
            ),
            AgentSkill(
                id="planning",
                name="Execution Planning",
                description="Create optimal execution plans for complex requests",
                tags=["planning", "strategy", "optimization"],
                inputModes=["text/plain"],
                outputModes=["application/json"]
            )
        ]

## 4. System Instruction for LLM

The system instruction guides the LLM on how to orchestrate agents effectively.

In [ ]:
    def get_system_instruction(self) -> str:
        """Provide detailed orchestration instructions to the LLM."""
        available_agents = ', '.join(self.agent_registry.keys())
        
        return f"""
You are an intelligent orchestrator that coordinates multiple specialized agents.

AVAILABLE AGENTS: {available_agents}

YOUR WORKFLOW:
1. Use 'understand_request' tool to parse the user's intent
2. Use 'plan_execution' tool to create an execution strategy
3. Use 'coordinate_agents' tool to manage agent calls:
   - Inside this tool, use self.call_agent() to call agents
   - Pass appropriate data between agents
   - Handle errors gracefully
4. Use 'synthesize_results' tool to combine outputs

IMPORTANT - Agent Communication:
- You call agents using: await self.call_agent(url, message)
- Agents return Artifacts (outputs), not Messages
- Each agent call is independent (no conversation)
- Pass context between agents as needed

ERROR HANDLING:
- If an agent fails, try alternatives or use partial results
- Always provide useful output even with failures
- Log issues for debugging

SYNTHESIS:
- Combine results from all agents coherently
- Highlight the most important findings
- Structure output clearly
- Provide actionable insights

Remember: You orchestrate by calling agents via call_agent(). 
The tools help you plan and synthesize, but actual coordination uses agent calls.
"""

## 5. Orchestration Tools - Understanding

Tools for understanding user requests. Copy and modify for your domain.

In [ ]:
def understand_request(
    request: str,
    context: Optional[Dict[str, Any]] = None
) -> str:
    """
    Parse and understand a user request.
    
    Args:
        request: The user's natural language request
        context: Optional context about the request
        
    Returns:
        JSON string with parsed intent and requirements
    """
    # This tool helps the LLM understand what the user wants
    understanding = {
        "original_request": request,
        "intent": "analyze",  # Will be determined by LLM
        "requirements": [],
        "priority": "normal",
        "context": context or {}
    }
    
    # Parse keywords to help guide understanding
    keywords = {
        "urgent": "high priority",
        "analyze": "analysis required",
        "validate": "validation needed",
        "process": "processing workflow",
        "report": "report generation"
    }
    
    for keyword, meaning in keywords.items():
        if keyword in request.lower():
            understanding["requirements"].append(meaning)
    
    return json.dumps(understanding, indent=2)

## 6. Orchestration Tools - Planning

Tools for planning execution strategies.

In [ ]:
def plan_execution(
    intent: Dict[str, Any],
    available_agents: List[str],
    constraints: Optional[Dict[str, Any]] = None
) -> str:
    """
    Create an execution plan for the request.
    
    Args:
        intent: Parsed intent from understand_request
        available_agents: List of available agent names
        constraints: Optional execution constraints (time, resources)
        
    Returns:
        JSON string with execution plan
    """
    # Create execution plan
    plan = {
        "strategy": "sequential",  # or "parallel", "conditional"
        "steps": [],
        "agents_to_use": [],
        "estimated_time": "30 seconds",
        "fallback_strategy": "use_partial_results"
    }
    
    # Build execution steps based on intent
    if "analysis" in str(intent):
        plan["steps"].append({
            "step": 1,
            "agent": "analyzer",
            "action": "analyze_data",
            "input": "user_request"
        })
        plan["agents_to_use"].append("analyzer")
    
    if "validation" in str(intent):
        plan["steps"].append({
            "step": 2,
            "agent": "validator",
            "action": "validate_results",
            "input": "analysis_output"
        })
        plan["agents_to_use"].append("validator")
    
    # Add processing step
    plan["steps"].append({
        "step": len(plan["steps"]) + 1,
        "agent": "processor",
        "action": "final_processing",
        "input": "all_results"
    })
    plan["agents_to_use"].append("processor")
    
    return json.dumps(plan, indent=2)

## 7. Orchestration Tools - Coordination

The main coordination tool that actually calls other agents.

In [ ]:
async def coordinate_agents(
    self,  # Need self to access call_agent method
    execution_plan: Dict[str, Any],
    initial_data: Any,
    parallel: bool = False
) -> str:
    """
    Execute the coordination plan by calling agents.
    
    This is where actual agent calls happen using self.call_agent().
    
    Args:
        self: The orchestrator instance (for call_agent access)
        execution_plan: The plan from plan_execution
        initial_data: Initial data to process
        parallel: Whether to run agents in parallel
        
    Returns:
        JSON string with coordination results
    """
    results = {
        "execution_id": f"exec_{datetime.now().timestamp()}",
        "plan": execution_plan,
        "agent_results": {},
        "errors": [],
        "status": "started"
    }
    
    try:
        if parallel and len(execution_plan.get("steps", [])) > 1:
            # Parallel execution
            tasks = []
            for step in execution_plan["steps"]:
                agent_name = step["agent"]
                agent_url = self.agent_registry.get(agent_name)
                
                if agent_url:
                    # Create async task for each agent
                    task = self.call_agent(agent_url, initial_data)
                    tasks.append((agent_name, task))
            
            # Wait for all agents
            for agent_name, task in tasks:
                try:
                    result = await task
                    results["agent_results"][agent_name] = result
                except Exception as e:
                    results["errors"].append({
                        "agent": agent_name,
                        "error": str(e)
                    })
        else:
            # Sequential execution with data passing
            current_data = initial_data
            
            for step in execution_plan.get("steps", []):
                agent_name = step["agent"]
                agent_url = self.agent_registry.get(agent_name)
                
                if not agent_url:
                    logger.warning(f"Agent {agent_name} not found in registry")
                    continue
                
                try:
                    # Call the agent with current data
                    logger.info(f"Calling {agent_name} at {agent_url}")
                    result = await self.call_agent(agent_url, current_data)
                    
                    # Store result
                    results["agent_results"][agent_name] = result
                    
                    # Pass result to next agent
                    current_data = result
                    
                except Exception as e:
                    logger.error(f"Error calling {agent_name}: {e}")
                    results["errors"].append({
                        "agent": agent_name,
                        "error": str(e),
                        "step": step.get("step")
                    })
                    # Continue with partial results
        
        results["status"] = "completed" if not results["errors"] else "partial"
        
    except Exception as e:
        logger.error(f"Coordination failed: {e}")
        results["status"] = "failed"
        results["errors"].append({"error": str(e)})
    
    return json.dumps(results, indent=2)

In [ ]:
def extract_from_artifact(response: Any) -> Any:
    """
    Extract data or text from artifact response structure.
    
    ALWAYS use this after calling an agent!
    
    Args:
        response: The response from call_agent()
        
    Returns:
        The actual data/text from the artifact
    """
    if isinstance(response, dict):
        # Check if this is an artifact with parts
        if "parts" in response and isinstance(response["parts"], list):
            for part in response["parts"]:
                if isinstance(part, dict):
                    if part.get("kind") == "data":
                        return part.get("data")
                    elif part.get("kind") == "text":
                        return part.get("text")
        
        # Check if response has artifactId and parts (full artifact)
        if "artifactId" in response and "parts" in response:
            return extract_from_artifact({"parts": response["parts"]})
    
    # Fallback - return as is
    return response


# CORRECT way to handle agent responses:
async def coordinate_agents_correct(self, plan, data):
    """Example showing correct artifact extraction."""
    results = {}
    
    for step in plan["steps"]:
        agent_url = self.agent_registry.get(step["agent"])
        
        # Call the agent
        artifact_response = await self.call_agent(agent_url, data)
        
        # CRITICAL: Extract data from artifact!
        actual_data = extract_from_artifact(artifact_response)
        
        # Now you have the actual data to work with
        results[step["agent"]] = actual_data
        
        # Pass extracted data to next agent, not the artifact
        data = actual_data
    
    return results

## 7a. CRITICAL: Extracting Data from Artifact Responses

⚠️ **IMPORTANT**: When you call agents via `call_agent()`, they return Artifacts, not raw data!

Artifacts have this structure:
```json
{
  "artifactId": "result-123",
  "parts": [
    {
      "kind": "data",
      "data": { /* your actual data here */ }
    }
  ]
}
```

You MUST extract the data from the artifact structure before processing.

## 8. Orchestration Tools - Synthesis

Tools for synthesizing results from multiple agents.

In [ ]:
def synthesize_results(
    coordination_results: Dict[str, Any],
    original_request: str,
    format: str = "structured"
) -> str:
    """
    Synthesize results from multiple agents into a coherent response.
    
    Args:
        coordination_results: Results from coordinate_agents
        original_request: The original user request
        format: Output format (structured, narrative, summary)
        
    Returns:
        JSON string with synthesized results
    """
    synthesis = {
        "request": original_request,
        "timestamp": datetime.now().isoformat(),
        "status": coordination_results.get("status", "unknown"),
        "summary": {},
        "detailed_results": {},
        "recommendations": [],
        "errors": coordination_results.get("errors", [])
    }
    
    # Process each agent's results
    agent_results = coordination_results.get("agent_results", {})
    
    for agent_name, result in agent_results.items():
        # Extract key information from each agent
        if isinstance(result, dict):
            synthesis["detailed_results"][agent_name] = result
            
            # Extract summary points
            if "summary" in result:
                synthesis["summary"][agent_name] = result["summary"]
            elif "result" in result:
                synthesis["summary"][agent_name] = result["result"]
            else:
                synthesis["summary"][agent_name] = "Processed successfully"
        else:
            synthesis["detailed_results"][agent_name] = str(result)
            synthesis["summary"][agent_name] = str(result)[:100] + "..."
    
    # Generate recommendations based on results
    if "analyzer" in agent_results:
        synthesis["recommendations"].append(
            "Analysis complete - review findings in detailed results"
        )
    
    if "validator" in agent_results:
        synthesis["recommendations"].append(
            "Validation performed - check for any issues"
        )
    
    # Add overall assessment
    if not synthesis["errors"]:
        synthesis["overall_assessment"] = "All agents completed successfully"
    else:
        synthesis["overall_assessment"] = f"Completed with {len(synthesis['errors'])} errors"
    
    return json.dumps(synthesis, indent=2)

## 9. Using Pydantic for Structured Orchestration

When orchestrating multiple agents, use Pydantic models to ensure consistent data structures across your coordination flow.

## 9. Error Handling Tools

Tools for handling errors and recovery strategies.

In [ ]:
def handle_errors(
    errors: List[Dict[str, Any]],
    partial_results: Dict[str, Any],
    recovery_strategy: str = "use_partial"
) -> str:
    """
    Handle errors and implement recovery strategies.
    
    Args:
        errors: List of errors from coordination
        partial_results: Any partial results obtained
        recovery_strategy: Strategy for recovery
        
    Returns:
        JSON string with error handling results
    """
    handling = {
        "errors_processed": len(errors),
        "recovery_strategy": recovery_strategy,
        "actions_taken": [],
        "partial_results_used": False,
        "fallback_response": None
    }
    
    # Analyze errors
    for error in errors:
        agent = error.get("agent", "unknown")
        error_msg = error.get("error", "")
        
        # Determine recovery action
        if "timeout" in error_msg.lower():
            handling["actions_taken"].append(
                f"Agent {agent} timed out - using cached or default values"
            )
        elif "connection" in error_msg.lower():
            handling["actions_taken"].append(
                f"Agent {agent} unreachable - skipping this step"
            )
        else:
            handling["actions_taken"].append(
                f"Agent {agent} failed - attempting to use partial results"
            )
    
    # Implement recovery
    if recovery_strategy == "use_partial" and partial_results:
        handling["partial_results_used"] = True
        handling["fallback_response"] = {
            "message": "Completed with partial results due to errors",
            "available_results": list(partial_results.keys())
        }
    elif recovery_strategy == "retry":
        handling["actions_taken"].append("Marked for retry in next execution")
    else:
        handling["fallback_response"] = {
            "message": "Unable to complete request due to errors",
            "suggestion": "Please try again or contact support"
        }
    
    return json.dumps(handling, indent=2)

## 10. Complete Orchestrator Implementation

Putting it all together into a complete orchestrator class.

In [ ]:
    # Add all tools to the orchestrator
    def get_tools(self) -> List[FunctionTool]:
        """
        Return orchestration tools for the LLM.
        These tools help plan and synthesize, while call_agent() does the actual work.
        """
        # Note: coordinate_agents needs self, so we bind it
        async def bound_coordinate_agents(execution_plan, initial_data, parallel=False):
            return await coordinate_agents(self, execution_plan, initial_data, parallel)
        
        return [
            FunctionTool(understand_request),
            FunctionTool(plan_execution),
            FunctionTool(bound_coordinate_agents),  # Bound to self
            FunctionTool(synthesize_results),
            FunctionTool(handle_errors)
        ]
    
    async def process_message(self, message: str) -> Union[str, Dict, List]:
        """
        Process orchestration request.
        Note: With tools, the LLM will handle the orchestration flow.
        """
        # This is typically not called when tools are provided
        # The LLM uses tools to orchestrate
        return "Orchestration handled via tools"

## 11. Status Update Retransmission

Advanced: Retransmit status updates from subagents to the user.

In [ ]:
    async def execute_with_updates(
        self,
        context,  # RequestContext
        event_queue  # EventQueue
    ):
        """
        Advanced execution with status update retransmission.
        This overrides the base execute method to add update forwarding.
        """
        from a2a.server.tasks import TaskUpdater
        from a2a.types import TaskState
        from a2a.utils import new_agent_text_message
        
        task = context.current_task
        updater = TaskUpdater(event_queue, task.id, task.context_id)
        
        # Start orchestration
        await updater.update_status(
            TaskState.working,
            new_agent_text_message("Starting orchestration...")
        )
        
        # Example: Call subagent with streaming updates
        agent_url = self.agent_registry.get("analyzer")
        if agent_url:
            try:
                # Create A2A client for streaming
                async with A2AClient(agent_url) as client:
                    # Start task on subagent
                    sub_task = await client.send_message(
                        {"role": "user", "parts": [{"kind": "text", "text": "analyze"}]}
                    )
                    
                    # Stream updates from subagent
                    async for update in client.stream_task(sub_task["id"]):
                        # Retransmit update to user
                        await updater.update_status(
                            TaskState.working,
                            new_agent_text_message(
                                f"Analyzer: {update.get('message', 'Processing...')}"
                            )
                        )
                        
                        # Check if complete
                        if update.get("status", {}).get("state") == "completed":
                            break
            
            except Exception as e:
                logger.error(f"Streaming failed: {e}")
        
        # Continue with normal execution
        await super().execute(context, event_queue)

## 12. Complete Orchestrator File

Here's the complete orchestrator as a single file:

In [ ]:
#!/usr/bin/env python3
"""
Complete Orchestrator Agent Implementation
Copy this entire cell to create your orchestrator.py file.
"""

import os
import sys
import json
import asyncio
from pathlib import Path
from typing import List, Dict, Any, Optional, Union
from datetime import datetime

sys.path.insert(0, str(Path(__file__).parent.parent))

from base import A2AAgent
from a2a.types import AgentSkill
from utils.logging import get_logger
from google.adk.tools import FunctionTool

logger = get_logger(__name__)


# Orchestration Tools
def understand_request(request: str, context: Optional[Dict] = None) -> str:
    """Parse user request."""
    return json.dumps({
        "request": request,
        "intent": "orchestrate",
        "context": context or {}
    })

def plan_execution(intent: Dict, available_agents: List[str]) -> str:
    """Plan execution strategy."""
    return json.dumps({
        "strategy": "sequential",
        "agents": available_agents,
        "steps": [{"agent": agent, "order": i} for i, agent in enumerate(available_agents)]
    })

def synthesize_results(results: Dict, request: str) -> str:
    """Synthesize agent results."""
    return json.dumps({
        "request": request,
        "synthesis": results,
        "status": "complete"
    })


class CompleteOrchestrator(A2AAgent):
    """Multi-agent orchestrator."""
    
    def __init__(self):
        super().__init__()
        self.agent_registry = {
            "analyzer": "http://analyzer:8000",
            "processor": "http://processor:8000"
        }
    
    def get_agent_name(self) -> str:
        return "Complete Orchestrator"
    
    def get_agent_description(self) -> str:
        return "Orchestrates multiple agents to complete complex tasks"
    
    def get_system_instruction(self) -> str:
        return """
        You orchestrate multiple agents. Use tools to:
        1. Understand the request
        2. Plan execution 
        3. Call agents via coordinate_agents tool
        4. Synthesize results
        """
    
    async def coordinate_agents_bound(self, plan: Dict, data: Any) -> str:
        """Coordinate agent execution."""
        results = {}
        for step in plan.get("steps", []):
            agent = step["agent"]
            url = self.agent_registry.get(agent)
            if url:
                try:
                    result = await self.call_agent(url, data)
                    results[agent] = result
                except Exception as e:
                    results[agent] = {"error": str(e)}
        return json.dumps(results)
    
    def get_tools(self) -> List[FunctionTool]:
        # Create bound coordinate function
        async def coordinate(plan, data):
            return await self.coordinate_agents_bound(plan, data)
        
        return [
            FunctionTool(understand_request),
            FunctionTool(plan_execution),
            FunctionTool(coordinate),
            FunctionTool(synthesize_results)
        ]
    
    async def process_message(self, message: str) -> str:
        return "Orchestration via tools"


# Module-level instance
orchestrator = CompleteOrchestrator()

## 13. Main Entry Point

The main.py for orchestrator is the same as for regular agents:

In [ ]:
# main.py - Same structure, just import the orchestrator
from orchestrator import CompleteOrchestrator

# Rest is identical to regular agent main.py
# The only difference is the import statement!

## Summary

You now have all the building blocks for an A2A-compliant orchestrator:

### Key Differences from Regular Agents:

1. **Tools for Orchestration**: Uses tools to understand, plan, coordinate, and synthesize
2. **Agent Registry**: Maintains a registry of available agents
3. **call_agent() Usage**: Actually calls other agents within tools
4. **Complex System Instructions**: Guides the LLM on orchestration workflow
5. **Error Recovery**: Handles partial failures gracefully
6. **Result Synthesis**: Combines outputs from multiple agents

### Orchestration Flow:

1. **Receive Request** (Message from user)
2. **Understand** intent using tools
3. **Plan** execution strategy
4. **Coordinate** agents (sequential or parallel)
5. **Handle** any errors
6. **Synthesize** results
7. **Return Artifact** with combined output

### To Build Your Orchestrator:

1. **Copy the cells** you need
2. **Modify agent registry** for your agents
3. **Customize tools** for your domain
4. **Adjust system instruction** for your workflow
5. **Save as orchestrator.py**
6. **Use same main.py** (just change import)

Remember: Orchestrators are agents too - they receive Messages and return Artifacts!

In [ ]:
# Define schemas for orchestration components
from pydantic import BaseModel, Field
from typing import Literal, List, Dict, Any
from datetime import timedelta

class AgentStep(BaseModel):
    """Single step in an execution plan."""
    step_number: int = Field(description="Step execution order")
    agent: str = Field(description="Agent name to call")
    action: str = Field(description="Action for the agent to perform")
    input_source: str = Field(description="Where to get input (user_request, previous_output, etc)")
    timeout: int = Field(default=30, description="Timeout in seconds")

class ExecutionPlan(BaseModel):
    """Complete execution plan for orchestration."""
    strategy: Literal["sequential", "parallel", "conditional"] = Field(
        description="Execution strategy"
    )
    steps: List[AgentStep] = Field(description="Steps to execute")
    estimated_time: str = Field(description="Estimated total execution time")
    fallback_strategy: str = Field(default="use_partial_results")

class CoordinationResult(BaseModel):
    """Results from agent coordination."""
    execution_id: str = Field(description="Unique execution identifier")
    plan: ExecutionPlan = Field(description="The plan that was executed")
    agent_results: Dict[str, Any] = Field(description="Results from each agent")
    errors: List[Dict[str, str]] = Field(default_factory=list, description="Any errors encountered")
    status: Literal["completed", "partial", "failed"] = Field(description="Overall status")
    
class SynthesizedResult(BaseModel):
    """Final synthesized result from orchestration."""
    request: str = Field(description="Original user request")
    summary: Dict[str, str] = Field(description="Summary from each agent")
    detailed_results: Dict[str, Any] = Field(description="Full results from each agent")
    recommendations: List[str] = Field(description="Action recommendations")
    overall_assessment: str = Field(description="Overall assessment of the execution")
    confidence: float = Field(ge=0, le=1, description="Confidence in the results")

### Using Pydantic in Orchestration Tools

Tools return JSON strings for the LLM, so use `.model_dump_json()` for serialization:

In [ ]:
def plan_execution_structured(
    request: str, 
    available_agents: List[str]
) -> str:
    """Create a structured execution plan using Pydantic."""
    
    # Build plan with Pydantic models
    steps = []
    
    # Add analyzer step
    if "analyzer" in available_agents:
        steps.append(AgentStep(
            step_number=1,
            agent="analyzer",
            action="analyze_request",
            input_source="user_request",
            timeout=30
        ))
    
    # Add validator step
    if "validator" in available_agents:
        steps.append(AgentStep(
            step_number=2,
            agent="validator",
            action="validate_analysis",
            input_source="previous_output",
            timeout=20
        ))
    
    # Create execution plan
    plan = ExecutionPlan(
        strategy="sequential",
        steps=steps,
        estimated_time="50 seconds",
        fallback_strategy="use_partial_results"
    )
    
    # Tools return JSON strings for the LLM
    return plan.model_dump_json(indent=2)


def synthesize_results_structured(
    coordination_data: str,
    original_request: str
) -> str:
    """Synthesize results using Pydantic models."""
    
    # Parse coordination results
    data = json.loads(coordination_data)
    
    # Build synthesized result
    result = SynthesizedResult(
        request=original_request,
        summary={
            agent: res.get("summary", "Completed") 
            for agent, res in data.get("agent_results", {}).items()
        },
        detailed_results=data.get("agent_results", {}),
        recommendations=[
            "Review analyzer findings",
            "Check validator results for issues"
        ],
        overall_assessment="Pipeline completed successfully",
        confidence=0.95 if not data.get("errors") else 0.7
    )
    
    # Return as JSON string for tool
    return result.model_dump_json(indent=2)

### Returning Structured Results from Orchestrator

When the orchestrator's `process_message` is called (if not using tools), return structured data:

In [ ]:
    async def process_message(self, message: str) -> Dict[str, Any]:
        """
        Process orchestration request with structured output.
        Note: Usually tools handle orchestration, but this shows manual approach.
        """
        
        # Manually orchestrate
        results = {}
        
        # Call agents
        for agent_name, agent_url in self.agent_registry.items():
            try:
                result = await self.call_agent(agent_url, message)
                results[agent_name] = result
            except Exception as e:
                results[agent_name] = {"error": str(e)}
        
        # Create structured result
        final_result = SynthesizedResult(
            request=message,
            summary={name: "Completed" for name in results.keys()},
            detailed_results=results,
            recommendations=["Review all agent outputs"],
            overall_assessment="Orchestration complete",
            confidence=0.9
        )
        
        # Return as dict for DataPart (A2A requirement)
        return final_result.model_dump()